In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dataset_name = 'cats_vs_dogs'
dataset, info = tfds.load(name=dataset_name, 
                          split=tfds.Split.TRAIN,
                          with_info=True)

In [3]:
def normalize_img(x):
  """Normalizes images: `uint8` -> `float32`."""
  x['image'] = tf.cast(x['image'], tf.float32) 
  x['image'] = x['image'] / 255
  x['image'] = tf.image.resize(x['image'], [224,224])
  return x

In [4]:
ds_train = dataset.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [5]:
for i in ds_train.take(1):
  print(i['image'].shape)
  print(i)

(224, 224, 3)
{'image': <tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.9457638 , 0.96929324, 0.96929324],
        [0.9245804 , 0.94149214, 0.9381833 ],
        [0.9015324 , 0.9140324 , 0.90692455],
        ...,
        [0.7828547 , 0.7235787 , 0.54390925],
        [0.71194637, 0.65883124, 0.47362408],
        [0.619661  , 0.5730803 , 0.38460588]],

       [[0.93343836, 0.95497197, 0.95397407],
        [0.9213378 , 0.9379377 , 0.9344729 ],
        [0.85643274, 0.8689327 , 0.86182487],
        ...,
        [0.7727624 , 0.71885836, 0.5389159 ],
        [0.73887867, 0.6912897 , 0.50542444],
        [0.709584  , 0.67015827, 0.47882363]],

       [[0.9291114 , 0.9447977 , 0.9408761 ],
        [0.8986678 , 0.9143541 , 0.9104325 ],
        [0.7813277 , 0.7948152 , 0.78474486],
        ...,
        [0.7423019 , 0.6946478 , 0.51651406],
        [0.7702698 , 0.72445405, 0.54412913],
        [0.7581304 , 0.7278783 , 0.53452605]],

       ...,

       [[0.637867  , 0.6087003 , 0.

In [6]:
ds_train = ds_train.shuffle(1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
for example in ds_train:
  train_image, train_label = example["image"], example["label"]

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [9]:
train_datagen.fit(train_image)

In [10]:
train_image[0]

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.3592787 , 0.14751402, 0.05339636],
        [0.3652608 , 0.15349609, 0.05158782],
        [0.36381233, 0.15204762, 0.04233123],
        ...,
        [0.01810224, 0.01810224, 0.01810224],
        [0.01807597, 0.01807597, 0.01807597],
        [0.01418067, 0.01418067, 0.01418067]],

       [[0.36862746, 0.15686275, 0.0627451 ],
        [0.34974217, 0.13797747, 0.0360692 ],
        [0.36723807, 0.15876257, 0.03917856],
        ...,
        [0.02685574, 0.02685574, 0.02685574],
        [0.02683346, 0.02683346, 0.02683346],
        [0.02352941, 0.02352941, 0.02352941]],

       [[0.3777311 , 0.16628152, 0.05553222],
        [0.3948774 , 0.18701012, 0.06551396],
        [0.4250806 , 0.21723746, 0.09179103],
        ...,
        [0.01924539, 0.01924539, 0.01140226],
        [0.01502764, 0.01502764, 0.00718872],
        [0.01081932, 0.01081932, 0.00360644]],

       ...,

       [[0.68438387, 0.44908974, 0.20595248],
        [0.71

**Model 1 Feature Extraction**

In [11]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [12]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [13]:
# Make all the layers in the pre-trained model non-trainable
for layer in base_model.layers:
  layer.trainable = False

In [14]:
last_layer = base_model.get_layer('out_relu')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 1280)


In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(2, activation='softmax')

In [16]:
x = global_average_layer(last_output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

In [17]:
model = tf.keras.Model(base_model.input, outputs)

In [18]:
from tensorflow.keras.optimizers import RMSprop
model.compile(
    optimizer=RMSprop(lr=0.0001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

In [19]:
train_image.shape

TensorShape([30, 224, 224, 3])

In [20]:
history=model.fit_generator(train_datagen.flow(train_image, train_label, batch_size=32), epochs=50)

Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7312 - accuracy: 0.4333
Epoch 2/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7407 - accuracy: 0.4000
Epoch 3/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7617 - accuracy: 0.5667
Epoch 4/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7727 - accuracy: 0.4000
Epoch 5/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7516 - accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7580 - accuracy: 0.4667
Epoch 7/50
1/1 [==============================] - 0s 2ms/step - loss: 0.7428 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7503 - accuracy: 0.4667
Epoch 9/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7823 - accuracy: 0.5000
Epoch 10/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7463 - accuracy: 0.4333
Epoch 11/50
1/1 [======================

In [21]:
model.save('cats.h5')

In [22]:
import os
print(str(os.path.getsize('./cats.h5')/1000000) + 'MB')

9.528032MB


**Model 2 Fine Tuning**

In [35]:
base_model.trainable = True

In [36]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  155


In [37]:
# Fine-tune from this layer onwards
fine_tune_at = 100

In [38]:
# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [39]:
from tensorflow.keras.optimizers import RMSprop
model.compile(
    optimizer=RMSprop(lr=0.0001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

In [40]:
len(model.trainable_variables)

58

In [41]:
history=model.fit_generator(train_datagen.flow(train_image, train_label, batch_size=32), epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 1ms/step - loss: 1.2989 - accuracy: 0.6000
Epoch 2/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7440 - accuracy: 0.4333
Epoch 3/50
1/1 [==============================] - 0s 1ms/step - loss: 1.3211 - accuracy: 0.6000
Epoch 4/50
1/1 [==============================] - 0s 1ms/step - loss: 0.7339 - accuracy: 0.4333
Epoch 5/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8519 - accuracy: 0.5667
Epoch 6/50
1/1 [==============================] - 0s 1ms/step - loss: 0.9528 - accuracy: 0.4000
Epoch 7/50
1/1 [==============================] - 0s 1ms/step - loss: 1.1027 - accuracy: 0.6000
Epoch 8/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8089 - accuracy: 0.4667
Epoch 9/50
1/1 [==============================] - 0s 1ms/step - loss: 1.0365 - accuracy: 0.6000
Epoch 10/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8144 - accuracy: 0.4000
Epoch 11/50
1/1 [======================

In [42]:
model.save('cats.h5')

In [43]:
import os
print(str(os.path.getsize('./cats.h5')/1000000) + 'MB')

17.097408MB
